<a href="https://colab.research.google.com/github/unni2888/NASA-Space-Apps-Challenge/blob/main/Neo_Earth_Close_Approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [80]:
!pip install -q kaggle

In [81]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"unnikrishnanck","key":"ffb7e48d6f0916d89ab25e1515aca518"}'}

In [82]:
# Creating a Kaggle directory
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [83]:
# Copy the Kaggle.json file to the created kaggle folder
!cp kaggle.json ~/.kaggle/

In [84]:
# Permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [85]:
# to list all datasets list
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                        Student Performance Factors                          94KB  2024-09-02 10:53:57          23758        434  1.0              
abdulszz/spotify-most-streamed-songs                           Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14           8438        107  1.0              
valakhorasani/mobile-device-usage-and-user-behavior-dataset    Mobile Device Usage and User Behavior Dataset        11KB  2024-09-28 20:21:12           1783         48  1.0              
valakhorasani/electric-vehicle-charging-patterns               El

In [86]:
! kaggle datasets download -d imtkaggleteam/nasa-neo-earth-close-approaches

Dataset URL: https://www.kaggle.com/datasets/imtkaggleteam/nasa-neo-earth-close-approaches
License(s): other
  0% 0.00/39.0k [00:00<?, ?B/s]
100% 39.0k/39.0k [00:00<00:00, 1.56MB/s]


In [87]:
!unzip nasa-neo-earth-close-approaches.zip

Archive:  nasa-neo-earth-close-approaches.zip
  inflating: NEO Earth Close Approaches.csv  


In [88]:
df = pd.read_csv("NEO Earth Close Approaches.csv")

In [90]:
df

,Object,Close-Approach (CA) Date,CA DistanceNominal (au),CA DistanceMinimum (au),V relative(km/s),V infinity(km/s),H(mag),Diameter,Rarity
0,(2023 JA1),2023-May-03 01:15 ± < 00:01,0.02518,0.02509,5.01,4.99,26.6,13 m - 29 m,0
1,(2023 HV5),2023-May-03 03:16 ± < 00:01,0.00291,0.00290,8.93,8.82,27.2,9.8 m - 22 m,0
2,(2023 JM2),2023-May-03 05:23 ± < 00:01,0.04008,0.03996,6.33,6.32,26.0,17 m - 38 m,0
3,(2023 HF7),2023-May-04 00:02 ± < 00:01,0.00427,0.00426,13.10,13.05,26.8,12 m - 26 m,0
4,(2023 HH7),2023-May-04 05:47 ± < 00:01,0.00561,0.00558,24.55,24.53,25.5,21 m - 47 m,1
...,...,...,...,...,...,...,...,...,...
1568,(2024 HO2),2024-Apr-29 15:18 ± 00:02,0.00025,0.00025,9.24,8.00,29.5,3.3 m - 7.5 m,1
1569,(2021 GD3),2024-Apr-30 05:31 ± < 00:01,0.02868,0.02868,3.52,3.50,27.0,11 m - 24 m,0
1570,(2024 HQ1),2024-Apr-30 14:03 ± < 00:01,0.00560,0.00559,5.52,5.43,28.3,5.8 m - 13 m,0
1571,(2022 TN1),2024-Apr-30 23:57 ± < 00:01,0.04766,0.04766,17.74,17.73,20.3,230 m - 520 m,1


In [91]:
df['Rarity'].value_counts()

,count
Rarity,
0,1399
1,149
2,24
3,1


In [92]:
max(df['V relative(km/s)'])

36.37

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573 entries, 0 to 1572
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Object                    1573 non-null   object 
 1   Close-Approach (CA) Date  1573 non-null   object 
 2   CA DistanceNominal (au)   1573 non-null   float64
 3   CA DistanceMinimum (au)   1573 non-null   float64
 4   V relative(km/s)          1573 non-null   float64
 5   V infinity(km/s)          1573 non-null   float64
 6   H(mag)                    1573 non-null   float64
 7   Diameter                  1573 non-null   object 
 8   Rarity                    1573 non-null   int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 110.7+ KB


In [94]:
X = df.drop(['Close-Approach (CA) Date', 'Rarity', 'Object', 'Diameter'], axis=1)
y = df['Rarity']

In [96]:
y = y.astype('category')

In [97]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1573 entries, 0 to 1572
Series name: Rarity
Non-Null Count  Dtype   
--------------  -----   
1573 non-null   category
dtypes: category(1)
memory usage: 1.9 KB


In [98]:
X

array([[0.50330728, 0.50413057, 0.12719176, 0.13410596, 0.58695652],
       [0.05692524, 0.05702196, 0.23629279, 0.23979029, 0.63043478],
       [0.80196432, 0.80374773, 0.16392986, 0.17080574, 0.54347826],
       ...,
       [0.11084386, 0.11122305, 0.14138603, 0.14624724, 0.71014493],
       [0.95389858, 0.95889583, 0.48149179, 0.48565121, 0.13043478],
       [0.38825416, 0.38827322, 0.49735597, 0.50137969, 0.52173913]])

In [100]:
y

,Rarity
0,0
1,0
2,0
3,0
4,1
...,...
1568,1
1569,0
1570,0
1571,1


In [101]:
X.shape, y.shape

((1573, 5), (1573,))

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1258, 5), (315, 5), (1258,), (315,))

In [104]:
X_train = MinMaxScaler().fit_transform(X_train)

# Random Forest

In [108]:
rf_model = RandomForestClassifier(n_estimators=100, random_state =42, class_weight = 'balanced')
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [109]:
rf_pred = rf_model.predict(X_test)

In [110]:
accuracy_score(y_test, rf_pred)

0.9555555555555556

In [111]:
confusion_matrix(y_test, rf_pred)

array([[278,   0,   0],
       [  9,  22,   0],
       [  0,   5,   1]])

In [112]:
dump(rf_model, open('NEO.pkl','wb'))

In [120]:
input = pd.DataFrame({"CA DistanceNominal (au)":0.00109, "CA DistanceMinimum (au)":0.00109, "V relative(km/s)":12.10, "V infinity(km/s)":11.90, "H(mag)":26.0}, index = [1])

In [121]:
input

,CA DistanceNominal (au),CA DistanceMinimum (au),V relative(km/s),V infinity(km/s),H(mag)
1,0.00109,0.00109,12.1,11.9,26.0


In [122]:
rf_model.predict(input)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([0])